In [58]:
#! pip3 install --upgrade pip
#! pip3 install --upgrade setuptools
#! pip3 -q install neuralcoref
#! pip3 uninstall spacy 
#! pip3 uninstall neuralcoref
#! pip3 install spacy==2.1.0 
#! pip3 install neuralcoref --no-binary neuralcoref

In [2]:
import spacy
import pickle
from collections import Counter





In [3]:
import neuralcoref

In [4]:
#! python3 -m spacy download en_core_web_lg

In [4]:
nlp = spacy.load('en_core_web_lg')

In [5]:
neuralcoref.add_to_pipe(nlp)

In [9]:
! ls 

coref.ipynb	      episodes_data.pickle		summaries.txt
coref_results.pickle  query_character_summaries.pickle	Summary.ipynb
CSI_fan_synopses      s01e07.csv
csi_USE.pickle	      summaries_query_mentions.pickle


In [6]:
import pickle
import pandas as pd
from os import listdir
import re


In [9]:
#d = pickle.load(open('episodes_data.pickle', 'rb'))
path_screenplays_scenes='csi-corpus/screenplay_summarization/scene_level_n_aspects'
all_chars = []
ep_characters = {}

    #ep_characters[ep] = [c.split(' ')[0] for c in d[ep]['characters']]

for ep in listdir(path_screenplays_scenes):
    #print(ep)
    annotated_scenes=pd.read_csv(path_screenplays_scenes+'/'+ep)
    #print(annotated_scenes.iloc[3].scene_text)
    print(ep)
    characters= annotated_scenes.scene_text.str.findall(r"\[\[([A-Za-z0-9_]+)\]\]")
    #set(re.findall(r"\[\[([A-Za-z0-9_]+)\]\]",s.str))
    #annotated_scenes.scene_text
    episode_characters= characters.explode().unique()
    episode_characters = [x for x in episode_characters if str(x) != 'nan']
    episode_characters_lastnames = [s.split('_')[-1] for s in episode_characters if not s.split('_')[-1].isdigit()]
    #print(episode_characters_lastnames)
    ep_characters[ep.split('.csv')[0]]= episode_characters_lastnames
    all_chars.extend(episode_characters_lastnames)
    

print(len(all_chars))
print(len(set(all_chars)))

s03e12.csv
s05e13.csv
s01e20.csv
s05e10.csv
s01e08.csv
s03e11.csv
s04e14.csv
s05e22.csv
s03e19.csv
s04e23.csv
s05e21.csv
s02e06.csv
s04e10.csv
s05e08.csv
s03e05.csv
s04e22.csv
s04e09.csv
s05e06.csv
s01e13.csv
s02e01.csv
s05e17.csv
s01e07.csv
s04e21.csv
s05e12.csv
s02e15.csv
s04e12.csv
s03e03.csv
s04e05.csv
s02e10.csv
s02e09.csv
s05e05.csv
s05e03.csv
s02e04.csv
s04e15.csv
s01e23.csv
s03e21.csv
s04e06.csv
s03e08.csv
s01e19.csv
966
406


In [10]:
full_text = {}
#files = [f for f in ]

path_synopses='CSI_fan_synopses'
d=listdir(path_synopses)
for ep in d:
    #print(ep)
    f = open(path_synopses+'/'+ep, "r")
    full_text[ep.split('.txt')[0]]=f.read().replace('\n', ' ')
    
    
full_text[ep.split('.txt')[0]]


'Synopsis A grisly collision between a car and an eighteen-wheeler leads to a murder investigation at a furry convention. Meanwhile, a man is found dead in a freezer in an apparent robbery-homicide.  Plot Victims: Linda Jones (deceased), Robert Pitt (deceased)  On the case: Catherine Willows, Gil Grissom, Warrick Brown, Jim Brass  On a dimly lit stretch of road, Linda Jones is driving and looking for something. She suddenly collides with something, causing a loud thud. As she looks back to see what she hit, she swerves into oncoming traffic, and collides with an 18-wheeler truck. Later that night, while officers investigate the truck driver, Grissom and Catherine follow Linda\'s tire tracks and see that she was braking for quite a while. They see a set of extremely large animal paw prints leading off the road, and find another victim: a man in a raccoon fursuit.  The next day, Grissom and Willows further investigate the "raccoon". Willows reveals that she once dated a Detroit Lions mas

In [8]:
#print(full_text['s01e07'])


In [9]:
ep_characters

{'s03e12': ['Man',
  'Grissom',
  'Catherine',
  'Sara',
  'Nick',
  'Robbins',
  'Phillips',
  'David',
  'Easton',
  'Easton',
  'Easton',
  'Warrick',
  'Gibson',
  'Linden',
  'Hinton',
  'Stearns',
  'Brass',
  'Marty',
  'Lawyer',
  'Menken',
  'Greg',
  'Handler',
  'Nurse'],
 's05e13': ['Foreman',
  'Cheryl',
  'Catherine',
  'Officer',
  'Warrick',
  'Nick',
  'Robbins',
  'Grissom',
  'Sara',
  'Greg',
  'Hodges',
  'Nurse',
  'Melton',
  'Ecklie',
  'Litre',
  'Vlad',
  'Brass',
  'Daniels',
  'Wellstone',
  'Melton',
  'Melton',
  'Chloe'],
 's01e20': ['Charlotte',
  'Lisa',
  'Sara',
  'Warrick',
  'Grissom',
  'Nick',
  'Catherine',
  'Brass',
  'Phillips',
  'Metcalf',
  'Officer',
  'Kendall',
  'Gilbert',
  'Halley',
  'Sanders',
  'Painter',
  'Dawson',
  'Walkey',
  'Gilbert',
  'Rucker',
  'Mark',
  'McCarty',
  'Brad',
  'Kendal'],
 's05e10': ['Brass',
  'Curtis',
  'Grissom',
  'Greg',
  'Officer',
  'Plummer',
  'Sara',
  'Phillips',
  'Man',
  'Warrick',
  'Nick

In [11]:
marked = {}
synopses_texts=listdir(path_synopses)
for ep in synopses_texts:
    ep=ep.split('.txt')[0]
    print(ep)
    marked[ep] = []
    episode_characters=ep_characters[ep]
    print(episode_characters)
    
    doc = nlp(full_text[ep])
    #print(doc)
    marks = []    
    for c in doc._.coref_clusters:
        for m in c.mentions:
            if c.main.text in episode_characters:
                marks.append(('#', m.start, c.main.text,  m.text))
            elif len(c.main.text.split(' ')) == 1 :
                marks.append(('~', m.start, c.main.text,  m.text))
        
        summary_chars = [l.text for l in doc]
        marks = sorted(marks, key=lambda x: x[1])
        
        #print(marks)
        
    for i, (m, ix, text, original) in enumerate(marks):
        at = ix
        try:
            summary_chars.pop(at)
            summary_chars.insert(at, m + text)
        except:
            print(i + ix)
        
        # print(' '.join(summary_chars))
    marked[ep].append(summary_chars)

s04e22
['Interrogator', 'Menlo', 'Brass', 'Grissom', 'Catherine', 'Phillips', 'number', 'Sara', 'Warrick', 'Croupier', 'Braun', 'Robbins', 'John', 'Bezich', 'Hodges', 'Nick', 'Player', 'Keller', 'Landers', 'Guard', 'Lawyer', 'Greene', 'Metcalf', 'Greg', 'Greene', 'Mullins']
s04e21
['Daughter', 'Joey', 'Mother', 'Voice', 'Lawrence', 'Brass', 'Grissom', 'Nick', 'Greg', 'Sara', 'Officer', 'Catherine', 'Phillips', 'Warrick', 'Woody', 'Press', 'Robbins', 'Anthony', 'Hunt', 'Henry', 'Press', 'Hodges', 'Brown', 'Mack', 'Tessa', 'Sidle', 'Worker']
s05e08
['Garner', 'Mann', 'Catherine', 'Grissom', 'Phillips', 'Warrick', 'Sara', 'Robbins', 'Brass', 'Johnson', 'Dickerson', 'Laner', 'Manager', 'Showgirl', 'Pink', 'Purple', 'Mimosa', 'Bev', 'Group', 'Greg', 'Hamshaw', 'Tippi', 'Francis', 'Ed', 'Dina', 'Lavalle', 'Nick', 'Bartender', 'Woman', 'Mercedes', 'Tippy']
s02e09
['Player', 'Dealer', 'Duncan', 'Security', 'Grissom', 'Brass', 'Warrick', 'Nick', 'Catherine', 'Sara', 'Spencer', 'Guy', 'Waitress'

s03e19
['Woman', 'Man', 'Audience', 'Audience', 'Usher', 'Catherine', 'Grissom', 'Brass', 'Robbins', 'Goer', 'Nick', 'Warrick', 'Sara', 'Greg', 'Franco', 'Archie', 'Hilden', 'Tech', 'Woman', 'McCallum', 'McCallum', 'Hodges', 'Haines', 'Goodson', 'Earl', 'Zachary', 'All', 'kid', 'JJ', 'Zach', 'Everyone']
s01e20
['Charlotte', 'Lisa', 'Sara', 'Warrick', 'Grissom', 'Nick', 'Catherine', 'Brass', 'Phillips', 'Metcalf', 'Officer', 'Kendall', 'Gilbert', 'Halley', 'Sanders', 'Painter', 'Dawson', 'Walkey', 'Gilbert', 'Rucker', 'Mark', 'McCarty', 'Brad', 'Kendal']
s05e05
['Wife', 'Husband', 'Brady', 'Guest', 'Guest', 'Brass', 'Grissom', 'Sara', 'Phillips', 'Keaton', 'Keaton', 'Greg', 'Nick', 'Warrick', 'Door', 'Gleason', 'Robbins', 'Pinscher', 'Hodges', 'Dickerson', 'Secretary', 'Brady', 'Poolie', 'Catherine', 'Cunningham', 'Cunningham']
s01e19
['Anderson', 'Grissom', 'Anderson', 'Catherine', 'Brass', 'Man', 'Anderson', 'Anderson', 'Sara', 'Nick', 'Warrick', 'Officer', 'Litre', 'Tech', 'Francis',

In [30]:
marked['s04e22']

[['Synopsis',
  'The',
  'CSIs',
  'investigate',
  'when',
  'two',
  'young',
  'gamblers',
  'are',
  'found',
  'dead',
  'shortly',
  'after',
  'scamming',
  'several',
  'of',
  'Sam',
  'Braun',
  "'s",
  'casinos',
  '.',
  '#Grissom',
  'is',
  'forced',
  'to',
  'take',
  '#Catherine',
  'off',
  'the',
  'case',
  'because',
  '#Catherine',
  'father',
  'is',
  'a',
  'suspect',
  'in',
  'their',
  'murder',
  '.',
  ' ',
  'Plot',
  'Victims',
  ':',
  'Teddy',
  'Keller',
  '(',
  'deceased',
  ')',
  ',',
  'Davis',
  'Mullins',
  '(',
  'deceased',
  ')',
  ' ',
  'On',
  'the',
  'case',
  ':',
  'Entire',
  'team',
  ' ',
  'In',
  'a',
  'dark',
  'room',
  ',',
  'a',
  'young',
  'man',
  'is',
  'questioned',
  'under',
  'a',
  'spotlight',
  'by',
  'two',
  'unseen',
  'people',
  '.',
  'One',
  'of',
  'the',
  'unseen',
  'people',
  'asks',
  'the',
  'young',
  'man',
  'if',
  'he',
  'knows',
  'what',
  'happened',
  'to',
  'people',
  'like',
  'hi

In [32]:
l = doc._.coref_clusters

In [ ]:
#[l.text for l in doc]

In [12]:
marked_2 = {}
for ep in marked:
    marked_2[ep] = []
    for i, tokens in enumerate(marked[ep]):
        episode_characters = ep_characters[ep]
        tokens_all = ['#' + token if token in episode_characters else token for token in tokens ]
        marked_2[ep].append(' '.join(tokens_all))

In [12]:
for ep in marked_2:
    print(ep, marked_2[ep])

s04e22 ['Synopsis The CSIs investigate when two young gamblers are found dead shortly after scamming several of Sam #Braun \'s casinos . #Grissom is forced to take #Catherine off the case because #Catherine father is a suspect in their murder .   Plot Victims : Teddy #Keller ( deceased ) , Davis #Mullins ( deceased )   On the case : Entire team   In a dark room , a young man is questioned under a spotlight by two unseen people . One of the unseen people asks the young man if he knows what happened to people like him back in the day . The young man is sweating bullets while this interrogation is going on . He insists that he it \'s illegal to hold him in the room ; the interrogator replies that there is no law in the room . The young man is later found dead in a neon graveyard with two shots to the back of the head . #Catherine , #Grissom and #Brass are on the scene ; #Grissom notices that the victim is wearing a wig and a fat suit . #Brass finds the victim \'s ID ; the victim \'s name 

In [13]:
sentences = {}
for ep in marked_2:
    episode_characters = ep_characters[ep]
    sentences[ep] = {'support':[], 'marked':[], 'characters':[]}
    for i, p in enumerate(marked_2[ep]):
        for j, sentence in enumerate(p.split(' . ')):
            sentences[ep]['support'].append(full_text[ep].split('.')[j].replace('\n', ' ').strip())
            sentences[ep]['marked'].append(sentence.replace('\n', ' ').strip())
            sentences[ep]['characters'].append([w[1:] for w in sentence.split(' ') if w[1:] in episode_characters])
            

                

In [24]:
print(sentences['s04e14']['marked'])

['Synopsis The entire team investigates when a robbery at a grocery store turns into a shootout and mass murder , leaving five people dead , one of which is a police officer', 'The evidence leads #Grissom to doubt the story of the other police officer at the scene', 'Plot Victim : Multiple victims ( deceased )   On the case : Entire team   A police car stops in front of the Best Bargain Supermarket', '#Officer David #Fromansky gives rookie #Officer #Clay a list of items to purchase', 'As #Officer #Clay is picking up the last item , he sees the reflection of a masked gunman in the ceiling mirror in the next aisle', 'The officer is last seen drawing his weapon', '#Catherine , #Grissom and #Warrick meet #Brass at the supermarket where #Brass gives a summary of the carnage', 'There are five victims - two unidentified gunmen , a woman being wheeled into an ambulance outside , cocktail waitress Julia Reed , and cab driver Rufus Sanders', "#Catherine snaps photos of Julia 's body , while #War

In [14]:
pickle.dump(sentences, open('summaries_mentions_first_story.pickle', 'wb'))

In [257]:
with open('summaries.txt', 'w') as f:
     for e in sentences:
        f.write(e + '\n')
        f.write('. '.join(s for s in sentences[e]['support']))
        f.write('\n\n')

In [234]:
summaries = {}
for ep in sentences:
    summaries[ep] = []
    for sen in sentences[ep]['marked']:
        l = []
        for w in sen.split(' '):
            if len(w) == 0: continue
            l.append(w[1:] if w[0] in ['#', '@', '~'] else w)
        summaries[ep].append(' '.join(l))

In [238]:
sum([len(summaries[ep])  for ep in summaries])

388

In [239]:
pickle.dump(summaries, open('query_character_summaries.pickle', 'wb'))

In [210]:
sum([len(sentences[ep]['support']) for ep in sentences for ep in sentences])

4268

In [181]:
counters = {}
for ep in sentences:
    all_chars = []
    all_chars.extend([k for l in sentences[ep]['characters'] for k in l])
    # all_chars.extend([k for l in sentences[ep]['queries'] for k in l])
    counters[ep] = Counter(all_chars).most_common()

In [182]:
counters

{'5531550228324592939.mp4': [('Ryan', 26),
  ('Stacey', 24),
  ('Janine', 20),
  ('Kat', 7),
  ('Pat', 5),
  ('Billy', 4),
  ('Kim', 2),
  ('Charlie', 2),
  ('Alfie', 2),
  ('Phil', 2),
  ('Jean', 1),
  ('Julie', 1),
  ('Bianca', 1)],
 '5534228999422914578.mp4': [('Janine', 20),
  ('Ryan', 20),
  ('Stacey', 16),
  ('Kat', 6),
  ('Pat', 4),
  ('Alfie', 2),
  ('Ricky', 1),
  ('Vanessa', 1),
  ('Jodie', 1),
  ('Max', 1),
  ('Julie', 1),
  ('Billy', 1)],
 '5539381671692122744.mp4': [('Stacey', 14),
  ('Max', 7),
  ('Lauren', 6),
  ('Kat', 3),
  ('Vanessa', 1)],
 '5542003749222140011.mp4': [('Stacey', 30),
  ('Lauren', 24),
  ('Max', 15),
  ('Ryan', 8),
  ('Phil', 3),
  ('Vanessa', 3),
  ('Janine', 1),
  ('Abi', 1),
  ('Jean', 1)],
 '5544574287152993687.mp4': [('Ryan', 25),
  ('Janine', 16),
  ('Kat', 13),
  ('Stacey', 7),
  ('Pat', 4),
  ('Alfie', 3),
  ('Max', 1),
  ('Glenda', 1),
  ('Phil', 1)],
 '5544620672795594434.mp4': [('Ryan', 25),
  ('Janine', 16),
  ('Kat', 13),
  ('Stacey', 7),


In [242]:
results =  {}
for ep in counters:
    results[ep] = []
    for first_name, count in counters[ep]:
        if count > 3:
            try:
                full_name = [n for n in d[ep]['characters'] if n.startswith(first_name)][0]
                results[ep].append((full_name, count))
            except:
                print(first_name, ep, d[ep]['characters'])

Stacey 5544574287152993687.mp4 ['Masood Ahmed', 'Jane Beale', 'Ian Beale', 'Zainab Masood', 'Janine Malloy', 'Ryan Malloy', 'Pat Evans', 'Phil Mitchell', 'Glenda Mitchell', 'Shirley Carter', 'Alfie Moon', 'Kat Moon', 'Bianca Butcher', 'Carol Jackson', 'Connor Stanley', 'Patrick Trueman', 'Billy Mitchell', 'Christian Clarke', 'Peter Beale', 'Tamwar Masood', 'Afia Khan', 'Yusef Khan', 'Ricky Butcher', 'Denise Johnson', 'Mo Harris', 'Whitney Dean', 'Jay Brown', 'Vanessa Gold', 'Lauren Branning', 'Max Branning', 'Roxy Mitchell', 'Heather Trott', 'Mr Atherton', 'Bushra Abbasi', 'Maria De Costa', 'Lad']
Stacey 5544620672795594434.mp4 ['Masood Ahmed', 'Jane Beale', 'Ian Beale', 'Zainab Masood', 'Janine Malloy', 'Ryan Malloy', 'Pat Evans', 'Phil Mitchell', 'Glenda Mitchell', 'Shirley Carter', 'Alfie Moon', 'Kat Moon', 'Bianca Butcher', 'Carol Jackson', 'Connor Stanley', 'Patrick Trueman', 'Billy Mitchell', 'Christian Clarke', 'Peter Beale', 'Tamwar Masood', 'Afia Khan', 'Yusef Khan', 'Ricky Bu

In [244]:
chars = set()
for ep in results:
    for c in [k[0] for k in results[ep]]:
        chars.add(c)
chars

{'Billy Mitchell',
 'Dot Branning',
 'Jane Beale',
 'Janine Malloy',
 'Jean Slater',
 'Julie Perkins',
 'Kat Moon',
 'Lauren Branning',
 'Max Branning',
 'Pat Evans',
 'Ronnie Branning',
 'Roxy Mitchell',
 'Ryan Malloy',
 'Stacey Branning'}

In [216]:
results

{'5531550228324592939.mp4': [('Ryan Malloy', 26),
  ('Stacey Branning', 24),
  ('Janine Malloy', 20),
  ('Kat Moon', 7),
  ('Pat Evans', 5),
  ('Billy Mitchell', 4),
  ('Kim Fox', 2),
  ('Charlie Slater', 2),
  ('Alfie Moon', 2),
  ('Phil Mitchell', 2),
  ('Jean Slater', 1),
  ('Julie Perkins', 1),
  ('Bianca Butcher', 1)],
 '5534228999422914578.mp4': [('Janine Malloy', 20),
  ('Ryan Malloy', 20),
  ('Stacey Branning', 16),
  ('Kat Moon', 6),
  ('Pat Evans', 4),
  ('Alfie Moon', 2),
  ('Ricky Butcher', 1),
  ('Vanessa Gold', 1),
  ('Jodie Gold', 1),
  ('Max Branning', 1),
  ('Julie Perkins', 1),
  ('Billy Mitchell', 1)],
 '5539381671692122744.mp4': [('Stacey Branning', 14),
  ('Max Branning', 7),
  ('Lauren Branning', 6),
  ('Kat Moon', 3),
  ('Vanessa Gold', 1)],
 '5542003749222140011.mp4': [('Stacey Branning', 30),
  ('Lauren Branning', 24),
  ('Max Branning', 15),
  ('Ryan Malloy', 8),
  ('Phil Mitchell', 3),
  ('Vanessa Gold', 3),
  ('Janine Butcher', 1),
  ('Abi Branning', 1),
  (

In [217]:
pickle.dump(results, open('coref_results.pickle', 'wb'))

In [ ]:
full_text.count('.')

In [12]:
text = """Kat persuades Stacey and Jean to come to R&R with her and Kim, leaving Charlie and Alfie to look after Lily. Alfie quickly escapes to the barrel store. There, Alfie fixes the new television to the wall, it falls off and smashes. Alfie joins the girls in R&R. Charlie has to pick up Mo and leaves Lily with Pat. Lily won’t settle so Pat takes her for a walk.
Janine accuses Ryan of wanting to go to R&R because he knew Stacey would be there. Ryan denies it and leaves, Janine stays in the club.
Julie explains that she was moved to another home when pregnant, their baby was taken away a week after she gave birth. Billy’s angry, first saying the baby should have stayed with them, then insisting Julie shouldn’t have told him about their son at all. Julie leaves, telling Billy he hasn’t changed.
Carol ends up in R&R, Billy joins her. The pair get drunk together. Carol cries, Billy tries to kiss her. When Carol rejects his advances, Billy insults her. Billy then unsuccessfully propositions Jean. Stacey realises Billy’s upset her mum, Janine and Kat get drawn into their argument before Alfie steps in. Ricky takes Carol home, Bianca storms to R&R to confront Billy. Giving Lily to Ryan, Pat follows Bianca into the club, worried. Bianca shouts at Billy. When Bianca backs off, Billy says she’s just like all the other ‘scrubbers’ there. Billy then turns on Stacey, who hits him. Janine pulls out a chunk of Stacey’s hair. Kat and Janine hit each other. Billy staggers away from the fight. Someone puts their hand on Billy’s shoulder, he instinctively turns round and hits them - unfortunately it’s a policeman. Roxy’s jealous when Christian brings Syed to R&R. Christian’s furious when Roxy deliberately spikes Syed’s drink."""

In [13]:
f
doc = nlp(text)

In [14]:
doc._.has_coref

True

In [28]:
for c in doc._.coref_clusters:
    for m in c.mentions:
        print(m, m.start, m.end)

Kat 0 1
her 10 11
Stacey 2 3
Stacey 91 92
Stacey 213 214
her 218 219
Stacey 301 302
R&R 8 9
R&R 87 88
he 89 90
R&R 176 177
R&R 242 243
Charlie 15 16
Charlie 54 55
Alfie 17 18
Alfie 23 24
Alfie 33 34
Alfie 48 49
Lily 21 22
Lily 62 63
Lily 66 67
her 73 74
the new television 35 38
it 42 43
Pat 64 65
Pat 71 72
Janine 79 80
Janine 102 103
she 112 113
she 129 130
Janine 221 222
Janine 295 296
Janine 307 308
Ryan 81 82
Ryan 96 97
Julie 109 110
Julie 149 150
Julie 161 162
their 121 122
them 145 146
their 156 157
their baby 121 123
the baby 139 141
him 154 155
their son 156 158
Billy 133 134
Billy 165 166
he 166 167
Billy 178 179
Billy 191 192
his 200 201
Billy 203 204
Billy 207 208
Billy then 207 209
Billy 215 216
Billy 245 246
Billy 264 265
Billy 271 272
Billy 285 286
him 293 294
Billy 312 313
Billy 324 325
he 328 329
Carol 172 173
her 180 181
Carol 188 189
her 195 196
Carol 198 199
her 205 206
Carol 236 237
her mum, Janine and Kat 218 224
their 227 228
Kat 223 224
Kat 305 306
Bianca 239 240
